In [84]:
# !pip install pandas pyxlsb openpyxl

In [85]:
import pandas as pd

In [ ]:
FILE_NAME = "abc"
SHEET_NAME = "def"

In [87]:
OUTPUT_FILE_NAME = "result.xlsx"

In [ ]:
df = pd.read_excel(
    FILE_NAME,
    sheet_name=SHEET_NAME,
    engine="pyxlsb",
    usecols="B:R",
    skiprows=6,
    nrows=7470,
)

In [ ]:
rows_with_total_sorted = df[
    df.apply(
        lambda row: row.astype(str).str.contains("total", case=False).any(), axis=1
    )
].sort_values(by="Grand Total", ascending=False)

In [91]:
result = pd.DataFrame(columns=rows_with_total_sorted.columns)

for index, row in rows_with_total_sorted.iterrows():
    # Extract the <a_number> from the 'cust_nbr' column in df2
    cust_nbr = str(row["cust_nbr"]).split()[0]

    # Find matching rows in df1
    matching_rows = df[
        (df["cust_nbr"].astype(str).str.split().str[0] == cust_nbr) & 
        (~df["cust_nbr"].astype(str).str.contains("total", case=False))
    ]
    
    to_concat_rows = []
    
    for sub_index, sub_row in matching_rows.iterrows():
        # Append the matching row to the result
        to_concat_rows.append(sub_row)
        
        # Check if there's a next row and modify the "cust_nbr" to empty string
        if sub_index + 1 < len(df):  # Ensure there's a next row
            next_row = df.iloc[sub_index + 1].copy()  # Get the next row
            if pd.isna(next_row.iloc[0]):
                to_concat_rows.append(next_row)  # Append the modified next row

    to_concat_df = pd.DataFrame(to_concat_rows, columns=rows_with_total_sorted.columns)

    # Append matching rows from df1 to the result
    result = pd.concat([result, to_concat_df], ignore_index=True)

    # Append the current row from df2
    result = pd.concat([result, pd.DataFrame([row])], ignore_index=True)

C:\Users\lyduc\AppData\Local\Temp\ipykernel_31236\3978194237.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, pd.DataFrame([row])], ignore_index=True)


In [93]:
result.to_excel(OUTPUT_FILE_NAME, index=False)